# LSTM model 1 on KaggleCompetition data set 

## Importing packages 

In [10]:
# Load libraries
import pandas as pd
import numpy as np
import os.path as op
import time
import matplotlib.pyplot as plt
#Loading tensorflow and keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from keras.utils import plot_model
#Loading nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,GlobalMaxPool1D,Dropout,concatenate
#Loading sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
#Loading K-folds
from sklearn.model_selection import StratifiedKFold
#Loading the model 
from sklearn.model_selection import train_test_split
import sys  
sys.path.insert(0, '../../../models')
from PY.model1 import get_model1

## Paths 

In [11]:
DATA_DIR = '../../../../../Data'
CODE_DIR_H5='../../models/H5'
# Create a path for the saving location of the model
MODEL_DIR_H5 = CODE_DIR_H5 + '/model1_Kaggle.h5'

## Loading Data 

In [12]:
# Load train data
train = pd.read_csv(op.join(DATA_DIR, 'Kaggle_competition/Claims_to_work_with/claims.csv'))
train

id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiumnews.com   
3                                Jessica Purkiss   
4                                 Howard Portnoy   
...                                          ...   
20795                              Jerome Hudson   
20796                           Benjamin Hoffman   
20797  Michael J. de la Merced and Rachel Abrams   
20798                                Alex Ansary   
20799                              David Swanson   

                                                    text  label  
0      House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1      Ever get the feeling your life circles the rou...      0  
2      Why the Truth Might Get You Fired October 29, ...      1  
3      Videos 15 Civilians Killed In Single US Airstr...      1  
4      Print \nAn Iranian woman has been sentenced to...      1  
...                                                  ...    ...  
20795  Rapper T. I. unloaded on black celebrities who...      0  
20796  When the Green Bay Packers lost to the Washing...      0  
20797  The Macy’s of today grew from the union of sev...      0  
20798  NATO, Russia To Hold Parallel Exercises In Bal...      1  
20799    David Swanson is an author, activist, journa...      1  

[20800 rows x 5 columns]

In [13]:
train = train[train['author'].notna()]

In [14]:
train=train.reset_index()

In [15]:
#Get the Depndent feature
X_train=train.drop('label',axis=1)
y_train=train['label']

In [16]:
size_comb=[0.05,0.1, 0.15, 0.2, 0.25, 0.3]
result_list=[]

In [17]:
#Main program 
for comb in (size_comb):
    test_size_value=comb
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=comb)
    
    # set vocabulary size
    vo_size=500
    messages=X_train.copy()
    messages.reset_index(inplace=True)
    
    # Dataset Preprocessing
    ps =PorterStemmer()
    corpus = []
    for i in range(0, len(messages)):
        print("Status: %s / %s" %(i, len(messages)), end="\r")
        review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
         
    # one hot representation
    onehot_rep = [one_hot(words, vo_size) for words in corpus]
    
    
    # pad_sequences
    sent_length = 22
    embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
    print(embedded_doc)
    embedding_vector_feature = 10
    
    
    messages1=X_test.copy()
    messages1.reset_index(inplace=True)

    
    
    # Dataset Preprocessing
    
    ps1 =PorterStemmer()
    corpus1 = []
    for i in range(0, len(messages1)):
        print("Status: %s / %s" %(i, len(messages1)), end="\r")
        review = re.sub('[^a-zA-Z]', ' ',messages1['title'][i])
        review = review.lower()
        review = review.split()
    
        review = [ps1.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus1.append(review)
        
        
    
    # one hot representation
    onehot_rep1 = [one_hot(words, vo_size) for words in corpus1]
    
    
    # pad_sequences
    sent_length = 22
    embedded_doc1=pad_sequences(onehot_rep1, padding='pre', maxlen=sent_length)
    print(embedded_doc1)
    
    
    # final data for NN
    X_final_train=np.array(embedded_doc)
    y_final_train=np.array(y_train)
    X_final_train.shape,y_final_train.shape
    
    X_final_test=np.array(embedded_doc1)
    y_final_test=np.array(y_test)
    X_final_test.shape,y_final_test.shape


    
    model = Sequential()
    model.add(Embedding(vo_size,embedding_vector_feature,input_length=sent_length))
    model.add(LSTM(100))
    model.add(Dense(1,activation='sigmoid')) 
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model 
    history=model.fit(X_final_train,y_final_train,validation_split=0.1,epochs=10,batch_size=64)
    # Evaluate the model 
    results = model.evaluate(X_final_test, y_final_test, batch_size=64)
    # Stock the results 
    item=(comb, y_final_train.shape, y_final_test.shape,results)
    result_list.append(item)

[[  0   0   0 ...  43 384 428]
 [  0   0   0 ... 237 252 252]
 [  0   0   0 ...  62 158 414]
 ...
 [  0   0   0 ... 127 280 219]
 [  0   0   0 ...  43 384 428]
 [  0   0   0 ...  37 348  11]]
[[  0   0   0 ... 464 210 453]
 [  0   0   0 ... 306 190 414]
 [  0   0   0 ...  43 384 428]
 ...
 [  0   0   0 ... 153 337 323]
 [  0   0   0 ...  43 384 428]
 [  0   0   0 ...  43 384 428]]
Train on 16110 samples, validate on 1790 samples
Epoch 1/10
16110/16110 [==============================] - 10s 592us/sample - loss: 0.3953 - accuracy: 0.8085 - val_loss: 0.2456 - val_accuracy: 0.8944
Epoch 2/10
16110/16110 [==============================] - 7s 429us/sample - loss: 0.2266 - accuracy: 0.9027 - val_loss: 0.2345 - val_accuracy: 0.8950
Epoch 3/10
16110/16110 [==============================] - 6s 380us/sample - loss: 0.2076 - accuracy: 0.9138 - val_loss: 0.2260 - val_accuracy: 0.9034
Epoch 4/10
16110/16110 [==============================] - 7s 425us/sample - loss: 0.1984 - accuracy: 0.9178 - val_lo

[[  0   0   0 ... 261  95 173]
 [  0   0   0 ... 185 355 395]
 [  0   0   0 ... 383  48 375]
 ...
 [  0   0   0 ... 404 242 365]
 [  0   0   0 ... 315 316 414]
 [  0   0   0 ...  43 384 428]]
Train on 7393 samples, validate on 822 samples
Epoch 1/10
7393/7393 [==============================] - 8s 1ms/sample - loss: 0.5171 - accuracy: 0.7264 - val_loss: 0.3520 - val_accuracy: 0.8431
Epoch 2/10
7393/7393 [==============================] - 4s 489us/sample - loss: 0.2855 - accuracy: 0.8780 - val_loss: 0.2499 - val_accuracy: 0.9015
Epoch 3/10
7393/7393 [==============================] - 4s 481us/sample - loss: 0.2323 - accuracy: 0.9055 - val_loss: 0.2367 - val_accuracy: 0.9015
Epoch 4/10
7393/7393 [==============================] - 4s 481us/sample - loss: 0.2147 - accuracy: 0.9092 - val_loss: 0.2374 - val_accuracy: 0.9015
Epoch 5/10
7393/7393 [==============================] - 4s 477us/sample - loss: 0.2066 - accuracy: 0.9122 - val_loss: 0.2378 - val_accuracy: 0.9027
Epoch 6/10
7393/7393 [=

In [18]:
result_list

[(0.05, (17900,), (943,), [0.23178967833518982, 0.9066808]),
 (0.1, (16110,), (1790,), [0.21127457803521077, 0.9134078]),
 (0.15, (13693,), (2417,), [0.22166638387645332, 0.9019446]),
 (0.2, (10954,), (2739,), [0.2610439181588957, 0.89485216]),
 (0.25, (8215,), (2739,), [0.2450100647495715, 0.8988682]),
 (0.3, (5750,), (2465,), [0.2987641630531808, 0.8892495])]